In [ ]:
%matplotlib auto



# Reading Data from invoice


In [ ]:
# Import required packages 
import cv2 
import pytesseract
import numpy as np
from openpyxl import Workbook
import unicodedata as ud
import re
import regex

file=input("Location of invoice")
# Mention the installed location of Tesseract-OCR 
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/cr712/AppData/Local/Tesseract-OCR/tesseract.exe'

# Read image from which text needs to be extracted 
if file.endswith('.jpg'):
    img = cv2.imread(file) 

    # Preprocessing the image starts 

    # Convert the image to gray scale 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    kernel = np.array([[0, -1, 0], 
                       [-1, 5,-1], 
                       [0, -1, 0]])

    # Sharpen image
    image_sharp = cv2.filter2D(gray, -1, kernel)

    # Performing OTSU threshold 
    ret, thresh1 = cv2.threshold(image_sharp, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV) 
    #ret, thresh1 = cv2.threshold(gray, 120, 255, cv2.THRESH_TOZERO_INV)

    # Specify structure shape and kernel size. 
    # Kernel size increases or decreases the area 
    # of the rectangle to be detected. 
    # A smaller value like (10, 10) will detect 
    # each word instead of a sentence. 
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30,50)) 

    # Appplying dilation on the threshold image 
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1) 

    # Finding contours 
    image, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 

    # Creating a copy of image 
    im2 = img.copy() 


    # A text file is created and flushed 
    data = open("outfile.txt", "w+") 
    data.write("") 
    data.close() 

    # Looping through the identified contours 
    # Then rectangular part is cropped and passed on 
    # to pytesseract for extracting text from it 
    # Extracted text is then written into the text file 
    i=1
    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2) 

        # Cropping the text block for giving input to OCR 
        cropped = im2[y:y + h, x:x + w] 

        # Open the file in append mode 
        data = open("outfile.txt", "a") 

        # Apply OCR on the cropped image 
        text = pytesseract.image_to_string(cropped) 
        print(i,'\n',text)
        i=i+1
        # Appending the text into file 
        data.write('\n') 
        data.write(text) 

        # Close the file 
        data.close 
else:
    if file.endswith('.pdf'):
        from PIL import Image 
        import sys 
        from pdf2image import convert_from_path 
        import os 

        # Path of the pdf 
        PDF_file = file

        # Store all the pages of the PDF in a variable 
        pages = convert_from_path(PDF_file, 500) 

        # Counter to store images of each page of PDF to image 
        image_counter = 1

        # Iterate through all the pages stored above 
        for page in pages: 

            filename = "page_"+str(image_counter)+".jpg"

            # Save the image of the page in system 
            page.save(filename, 'JPEG') 

            # Increment the counter to update filename 
            image_counter = image_counter + 1

        
       

        # Variable to get count of total number of pages 
        filelimit = image_counter-1

         
        f = open('outfile.txt', "w+") 

        
        for i in range(1, filelimit + 1): 

           
            filename = "page_"+str(i)+".jpg"

             
            text = str(((pytesseract.image_to_string(Image.open(filename))))) 

           
            f.write(text) 

        
        f.close() 

# Function for writing to excel file

In [ ]:
def writeToXL(invoice_no,address,currency,total):
    wb=Workbook()

    ws=wb.active
    
    ws.cell(row=1,column=1).value = 'INVOICE NUMBER'
    ws.cell(row=1,column=2).value = 'ADDRESS'
    ws.cell(row=1,column=3).value = 'CURRENCY'
    ws.cell(row=1,column=4).value = 'TOTAL'
    
    ws.cell(row=2,column=1).value = invoice_no
    ws.cell(row=2,column=2).value = address
    ws.cell(row=2,column=3).value = currency
    ws.cell(row=2,column=4).value = total
    wb.save('Invoice_data.xlsx')

# Extracting Data and Writing it to Excel Sheet

In [ ]:

#detecting currency
data=open('outfile.txt','r')
data=str(data.read())
'''cur=regex.findall(r'\p{Sc}',data)
sign=ud.name(cur[0])
currency=sign.replace('SIGN',"")'''
#print(currency)

#detecting adress
#Check Keyword if present 
key_check=re.compile(r'(?:Payable *\s*|Billed *to\s*|Shipping Address\s*|Shipped to\s*|Client name\s*).*\n.*\n?.*',re.I|re.MULTILINE)
address=key_check.findall(data)
address=address[0]
#print(address)

#detecting invoice no
number_regex=re.compile(r'(?:invoice *no\s*|invoice *number\s*|ref\s*)\.*(?:\#)*(?:\:)* *(\S*)',re.I|re.MULTILINE)
invoice_no=number_regex.findall(text)
invoice_no=invoice_no[0]
#print(invoice_no)


total=re.compile(r"(?:Total)(?:\s*)(?:\$)*(?:\s*)(\d+.*)",re.I|re.MULTILINE)
total=total.findall(data)
total=str(total[0])     
#print(total)

writeToXL(invoice_no,address,'currency',total)
print('Saved as Invoice_data.xlsx in current working directory')
